## Librerias

In [41]:
import numpy as np
import random
import copy
import collections
import PySimpleGUI as sg
import csv
import pandas
import time
from itertools import chain

# Distancias 

In [42]:
def distanciaHamming(cadena_uno, cadena_dos):
    distancia = 0
    for i in range(len(cadena_uno)):
        if cadena_uno[i] != cadena_dos[i]:
            distancia = distancia + 1
    return distancia

In [43]:
def distanciaEuclidiana(cadena_uno, cadena_dos):
    distancia = 0
   
    for i in range(len(cadena_uno)):
        distancia = distancia + ((cadena_uno[i] - cadena_dos[i])**2)
    distancia = np.sqrt(distancia)
    ##print(cadena_uno, cadena_dos, distancia)
    return distancia

# Operadores de selección

### Algoritmo de selección NAM

In [44]:
def NAM (poblacion, evaluaciones, funcionDistancia = 0, numero_padres = 0, tam_torneo = 0, maximizar=True):
    #evaluaciones = evaluaciones.astype(int)
    padres = []
    posicion_padre_uno = np.random.randint(len(poblacion))-1
    padres.append(poblacion[posicion_padre_uno]) 
    lejano = 0
    for i in range(len(poblacion)):
        distancia =  eval(funcionDistancia + '(poblacion[posicion_padre_uno], poblacion[i])')
        if distancia > lejano:
            padre_dos = poblacion[i]
            lejano = distancia
    padres.append(padre_dos)
    return padres

### Algoritmo de selección proporcional

In [45]:
def proporcional (poblacion, evaluaciones, funcionDistancia = 0, numero_padres = 0, tam_torneo = 0, maximizar=True):
    probabilidades = []
    padres = []
    sumatoria = np.sum(evaluaciones)
    for i in range(len(evaluaciones)):
        if maximizar == True:
            probabilidades.append(evaluaciones[i]/sumatoria)##1-(i/sumatoria) para min
        else:
            probabilidades.append(1-(evaluaciones[i]/sumatoria))
    for i in range(len(probabilidades)-1):
        probabilidades[i+1] = probabilidades[i] + probabilidades[i+1]
    for i in range(numero_padres):
        valor = np.random.uniform(0,1)
        bandera = 1
        j = 0
        while bandera == 1:
            if valor < probabilidades[j]:
                padres.append(poblacion[j])
                bandera = 0
            j = j + 1
    return padres 

### Algoritmo de selección por torneo 

In [46]:
def torneo (poblacion, evaluaciones, funcionDistancia = 0, numero_padres = 0, tam_torneo = 0, maximizar=True):
    padres = []
    
    for i in range(numero_padres):
        torneo = []
        sujetos = []
        for j in range(tam_torneo):
            posicion = (np.random.randint(0,len(evaluaciones)))
            sujetos.append(poblacion[posicion])
            torneo.append(evaluaciones[posicion])
            
        padres.append(sujetos[np.argmax(torneo)])
        
    return padres

# Operadores de Cruzamiento

###  Cruza en un punto

In [47]:
def cruza1Punto(padres, cruze):
    hijos = []
    i = 0
    while i < len(padres):
        pareja = copy.copy(padres)
        hijo1 = np.append(pareja[0][:cruze], pareja[1][cruze:])
        hijo2 = np.append(pareja[1][:cruze], pareja[0][cruze:])
        i = i + 2
        hijos.append(hijo1)
        hijos.append(hijo2)
    return hijos

###  Cruza en 'N' puntos

In [48]:
def cruzaNPuntos(padres, puntos = 0, num_hijos=0):
    cruces = []
    hijos = copy.copy(padres)
    
    while len(hijos) < num_hijos:
        while len(cruces) < puntos:
            punto = random.randint(1,len(padres[0]))
            if punto not in cruces:
                cruces.append(punto)

        for i in cruces:
            hijos = cruza1Punto(hijos, i)
    return hijos

###  Cruza uniforme

In [49]:
def cruzaUniforme(padres, puntos = 0, num_hijos=0):
    hijos = []
    i = 0
    while i < len(padres):
        pareja = copy.copy(padres)
        hijo1 = []
        hijo2 = []
        for j in range(len(padres[0])):
            selec = random.randint(0,1)
            #print(selec)
            hijo1 = np.append(hijo1, pareja[selec][j])
            hijo2 = np.append(hijo2, pareja[abs(selec-1)][j])
        i = i+2
        hijos.append(hijo1)
        hijos.append(hijo2)
        #print(pareja)
        #print(hijo1)
        #print(hijo2)
    return hijos

### Cruzamiento ordenado (OX)

In [50]:
def cruzaOrdenada(padres, puntos = 0, num_hijos=2):
    hijos = np.full((num_hijos,len(padres[0])-1), -1)

    # Eliminar ciudad repetida
    for j in range(len(padres)):
        padres[j] = padres[j][:-1]
    # Creación de un hijo
    for i in range(num_hijos):
        posiciones = np.sort(np.random.choice(list(range(0, len(padres[1])+1)), 2, replace=False))
        subtour = padres[0][posiciones[0]:posiciones[1]]

        hijos[i] = np.full(len(padres[0]), -1)
        

        hijos[i][posiciones[0]:posiciones[1]] = subtour
        # Completar el hijo
        for idx in range(len(padres[0])):
            ciudad = padres[1][idx]
            if ciudad not in hijos[i]:
                bandera = True
                j = 0
                while bandera == True:
                    if hijos[i][j] == -1:
                        hijos[i][j] = ciudad
                        bandera = False
                    else:
                        j = j+1
        padres[0], padres[1] = padres[1], padres[0]
    hijos2 = []
    for i in range(len(hijos)):
        hijo = np.append(hijos[i],hijos[i][0])
        hijos2.append(hijo)
    return hijos2


### Cruzamiento Aritmético

In [51]:
def cruza_aritmetica(padres, puntos = 0, num_hijos=0):
  hijos = np.full((2,len(padres[0])), -1, dtype='float64')
  alpha = np.random.uniform(0,1)
  for i in range(len(padres[0])):
    hijos[0][i] = (alpha*padres[0][i]) + ((1-alpha)*padres[1][i])
    hijos[1][i] = (alpha*padres[1][i]) + ((1-alpha)*padres[0][i])
  return hijos

# Operadores de Mutación

### Swap (EM)

In [52]:
def swap(hijo, probabilidad_m):
  prob_ind = np.random.random()
  if prob_ind <= probabilidad_m:
    hijo = hijo[0:-1]
    posiciones = np.sort(np.random.choice(list(range(0, len(hijo))), 2, replace=False))
    hijo_mutante = copy.copy(hijo)
    hijo_mutante[posiciones[0]] = hijo[posiciones[1]]
    hijo_mutante[posiciones[1]] = hijo[posiciones[0]]
    hijo_mutante = np.append(hijo_mutante,hijo_mutante[0])
    print('Posiciones = ',posiciones)
  else:
    hijo_mutante = copy.copy(hijo)
  return hijo_mutante


### Swap (EM) por gen

In [53]:
def swap_por_gen(hijo, probabilidad_m):
  hijo = hijo[0:-1]
  hijo_mutante = copy.copy(hijo)
  for gen1 in range(len(hijo)):
    prob_gen = np.random.random()
    if prob_gen <= probabilidad_m:
      gen2 = np.random.randint(gen1,len(hijo))
      hijo_mutante[gen1], hijo_mutante[gen2] = hijo_mutante[gen2], hijo_mutante[gen1]
  hijo_mutante = np.append(hijo_mutante, hijo_mutante[0])
  return hijo_mutante

### Inversión Simple (SIM)

In [54]:
def sim(hijo, probabilidad_m):
  prob_ind = np.random.random()
  if prob_ind <= probabilidad_m:
    hijo = hijo[0:-1]
    posiciones = np.sort(np.random.choice(list(range(0, len(hijo))), 2, replace=False))
    hijo_mutante = copy.copy(hijo)
    subtour = hijo[posiciones[0]:posiciones[1]+1]
    subtour = subtour[::-1]
    hijo_mutante[posiciones[0]:posiciones[1]+1] = subtour
    hijo_mutante = np.append(hijo_mutante,hijo_mutante[0])
  else:
    hijo_mutante = copy.copy(hijo)
  return hijo_mutante

### Inversión Simple (SIM) por gen

In [55]:
def sim_por_gen(hijo, probabilidad_m):
  hijo = hijo[0:-1]
  hijo_mutante = copy.copy(hijo)
  for pos_in in range(len(hijo)):
    prob_gen = np.random.random()
    if prob_gen <= probabilidad_m:
      pos_final = np.random.randint(pos_in, len(hijo))
      subtour = hijo_mutante[pos_in:pos_final]
      subtour = subtour[::-1]
      hijo_mutante[pos_in:pos_final] = subtour
  hijo_mutante = np.append(hijo_mutante, hijo_mutante[0])
  return hijo_mutante

### Mutación Uniforme

In [56]:
def uniform_mutation(hijo, rango, probabilidad_m):
  hijo_mutado = copy.copy(hijo)
  for posicion in range(len(hijo)):
    prob_gen = np.random.random()
    if prob_gen <= probabilidad_m:
      gen_mutado = np.random.uniform(rango[0], rango[1])
      hijo_mutado[posicion] = gen_mutado
  return hijo_mutado

### Mutación Límite

In [57]:
def boundary_mutation(hijo, rango, probabilidad_m):
  hijo_mutante = copy.copy(hijo)
  for posicion in range(len(hijo)):
    prob_gen = np.random.random()
    if prob_gen <= probabilidad_m:
      prob_limite = np.random.random()
      if prob_limite <= 0.5:
        gen_mutado = rango[0]
        hijo_mutante[posicion] = gen_mutado
      else:
        gen_mutado = rango[1]
        hijo_mutante[posicion] = gen_mutado
  return hijo_mutante

### Mutacion binaria

In [58]:
def mutacion_bin(hijo, probabilidad_m):
  hijo_mutante = copy.copy(hijo)
  for posicion in range(len(hijo)):
    prob_gen = np.random.random()
    gen = hijo[posicion]
    if ((gen == 1) and (prob_gen <= probabilidad_m)):
      gen_mutado = 0
      hijo_mutante[posicion] = gen_mutado
    elif ((gen == 0) and (prob_gen <= probabilidad_m)):
      gen_mutado = 1
      hijo_mutante[posicion] = gen_mutado
  return hijo_mutante

# Operadores de Reemplazo

### Random

In [59]:
def reemplazo_rand(poblacion, hijos, valores=0, padres=0, f_evaluacion=0, f_distancia=0, n=0):
  posiciones = np.random.choice(list(range(0, len(poblacion))), 2, replace=False)
  poblacion[posiciones[0]] = hijos[0]
  poblacion[posiciones[1]] = hijos[1]
  return poblacion

### Crowding Determinista

In [60]:
def reemplazo_CD(poblacion, hijos, valores=0, padres=0, f_evaluacion=0, f_distancia=0, n=0):
  eval_padres = []
  eval_hijos = []
  distancias = []
  newpoblacion = copy.copy(poblacion)
  for individuo in padres:
    eval_padre = f_evaluacion(individuo, valores)
    eval_padres.append(eval_padre)
  for individuo in hijos:
    eval_hijo = f_evaluacion(individuo, valores)
    eval_hijos.append(eval_hijo)
  for padre in padres:
    for hijo in hijos:
        distancia = eval(f_distancia + '(padre,hijo)')
        distancias.append(distancia)
      #compite p1-> h1 & p2->h2
  if (distancias[0] + distancias[3]) <= (distancias[1] + distancias[2]):
    if eval_padres[0] < eval_hijos[0]:
      for i in range(len(poblacion)):
        if (poblacion[i] == padres[0]).all() == True:
          newpoblacion[i] = hijos[0]
    elif eval_padres[1] < eval_hijos[1]:
      for i in range(len(poblacion)):
        if (poblacion[i] == padres[1]).all() == True:
          newpoblacion[i] = hijos[1]
  else:
    if eval_padres[0] < eval_hijos[1]:
      for i in range(len(poblacion)):
        if (poblacion[i] == padres[0]).all() == True:
          newpoblacion[i] = hijos[1]
    elif eval_padres[1] < eval_hijos[0]:
      for i in range(len(poblacion)):
        if (poblacion[i] == padres[1]).all() == True:
          newpoblacion[i] = hijos[0]

  
  return newpoblacion

### Torneo Restringido (TRS)

In [61]:
def TRS(poblacion, hijos, valores=0, padres=0, f_evaluacion=0, f_distancia=0, n=0):
  eval_hijos = []
  eval_individuos = []
  # Evaluacio de hijos
  for individuo in hijos:
    eval_hijo = f_evaluacion(individuo, valores)
    eval_hijos.append(eval_hijo)
  # n individuos
  posiciones = np.random.choice(list(range(0, len(poblacion))), n, replace=False)

  individuos = []
  for hijo in hijos:
    distancia = 9000
    for i in posiciones:
      dtemp = eval(f_distancia + '(hijo,poblacion[i])')
      if dtemp < distancia:
        distancia = dtemp
        pos = i
    individuos.append(pos)

  # Evaluacio de individuos cercanos
  for individuo in individuos:
    eval_individuo = f_evaluacion(poblacion[individuo], valores)
    eval_individuos.append(eval_individuo)

  # Comparar
  if eval_hijos[0] > eval_individuos[0]:
    poblacion[individuos[0]] = hijos[0]
  elif eval_hijos[1] > eval_individuos[1]:
    poblacion[individuos[1]] = hijos[1]

  return poblacion

# Generadores de Población

### Poblacion para el problema de la mochila 

In [62]:
def PesoTotal(solucion,pesos,pesoM):
    pesoT = 0
    for i in range(len(solucion)):
        if solucion[i] == 1:
            pesoT = pesoT + pesos[i]
    
    # Verificar si la solucion es valida, si el peso cabe en la mochila
    if pesoT <= pesoM:
        posible = True
    else:
        posible = False
    return posible

In [63]:
def poblacionMochila (sujetos, dimension, pesos, pesoM):
    poblacion = []
    while len(poblacion) < sujetos:
        sujeto = np.random.randint(2,size=(dimension))
        pesoT = PesoTotal(sujeto,pesos,pesoM)
        if pesoT == True:
            poblacion.append(sujeto)
    poblacion = np.array(poblacion)
    return poblacion

In [64]:
valores = [1,5,9,10,5]
poblacion = poblacionMochila(5,len(valores),valores,10)
print(poblacion)

[[1 0 1 0 0]
 [0 0 1 0 0]
 [1 0 1 0 0]
 [0 0 0 0 1]
 [1 0 0 0 0]]


### Población para el problema de minimos

In [65]:
def poblacionMinimos(sujetos, dimension,rango=[-10, 10]):
    poblacion = np.zeros((sujetos,dimension))
    for i in range(sujetos):
        for j in range(dimension):
            poblacion[i][j] = random.uniform(rango[0],rango[1])
    return poblacion

### Poblacion para el problema del viajero 

In [66]:
def poblacionViajero (sujetos, ciudades):
    poblacion = np.zeros((sujetos,len(ciudades)))
    for i in range(sujetos):
        poblacion[i] = copy.copy(ciudades) 
        random.shuffle(poblacion[i])
    camino_real = poblacion[:,0]
    camino_real = camino_real.reshape((sujetos,1))
    poblacion = np.concatenate((poblacion,camino_real), axis=1) 
    poblacion = poblacion.astype(int)
    return poblacion

# Evaluaciones

### Evaluación de poblacion para el problema del viajero  

In [67]:
def evaluacionViajero (poblacion, valores):
    distancia = np.zeros(len(poblacion))
    for i in range(len(poblacion)):
        for j in range(len(valores)):
            ciudad1 = poblacion[i][j]
            ciudad2 = poblacion[i][j+1]
            ciudad1 = int(ciudad1)-1
            ciudad2 = int(ciudad2)-1
            distancia[i] = distancia[i] + valores[ciudad1][ciudad2]
    return distancia

### Individuo viajero

In [68]:
def ind_viajero(camino, matriz_ady):
    distancia = 0

    for i in range(len(camino)-1):
        ciudad1 = camino[i]-1
        ciudad2 = camino[i+1]-1
        distancia = distancia + matriz_ady[ciudad1,ciudad2]

    return distancia

### Evaluación de población del problema de la mochila

In [69]:
def evaluacionMochila (poblacion, valores):
    evaluaciones = np.zeros(len(poblacion))
    for i in range(len(poblacion)):
        for j in range(len(poblacion[0])):
            if poblacion [i,j] == 1:
                evaluaciones[i] = evaluaciones[i] + valores[j] 
    return evaluaciones  

### Individuo Mochila

In [70]:
def ind_mochila(solucion, valores):
    valorT = 0
    for i in range(len(solucion)):
        if solucion[i] == 1:
            valorT = valorT + valores[i]
    return valorT

### Solución Posible Mochila

In [71]:
def verif_mochila(solucion,pesos,pesoM):
    pesoT = 0
    for i in range(len(solucion)):
        if solucion[i] == 1:
            pesoT = pesoT + pesos[i]
    
    # Verificar si la solucion es valida, si el peso cabe en la mochila
    if pesoT <= pesoM:
        posible = True
    else:
        posible = False
    return posible


### Evaluación de población para el problema del minimos cuadrado 

In [72]:
def evaluacionMinimos (poblacion, valores=0):
    evaluacion = np.zeros(len(poblacion)) 
    for i in range(len(poblacion)):
        for j in range(len(poblacion[0])):
            evaluacion[i] = evaluacion[i] + poblacion[i][j] ** 2
    return evaluacion

### Individuo minimos

In [73]:
def ind_minimos(individuo, valores=0):
    valor = 0
    for i in individuo:
        valor = valor + i
    return valor

### Funciones

In [74]:
# Alpine 1 Function
def Alpine1(solucion):
    fx = 0
    for x in solucion:
        fx = fx + abs((x * np.sin(x)) + (0.1 * x))
    return fx


# Dixon & Price Function
def Dixon(solucion):
    dimensiones = len(solucion)
    sumatoria = 0
    termino_1 = np.square((solucion[0] - 1))
    for i in range(1,dimensiones):
        x = solucion[i]
        pos = i + 1
        sumatoria = sumatoria + (pos*np.square((2*np.sin(x)) - solucion[i-1]))
    fx = termino_1 + sumatoria
    return fx


# Quintic Function
def Quintic(solucion):
    fx = 0
    for x in solucion:
        fx = fx + abs(np.power(x,5) - (3*np.power(x,4)) + (4*np.power(x,3)) - (2*np.power(x,2)) - (10*x) - 4)
    return fx


# Schwefel 2.23 Function
def Schwefel(solucion):
    fx = 0
    for x in solucion:
        fx = fx + np.power(x,10)
    return fx


# Streched V Sine Wave Function
def Streched(solucion):
    dimensiones = len(solucion)
    fx = 0
    for i in range(dimensiones - 1):
        x1 = solucion[i]
        x2 = solucion[i+1]
        termino_1 = np.power((np.square(x2) + np.square(x1)),0.25)
        termino_2 = np.square(np.sin((50*np.power((np.square(x2)+np.square(x1)),0.1)))) + 0.1
        interno = termino_1 * termino_2
        fx = fx + interno
    return fx


# Sum Squares Function
def SumSquares(solucion):
    dimensiones = len(solucion)
    fx = 0
    for i in range(dimensiones):
        pos = i + 1
        fx = fx + (pos * np.square(solucion[i]))
    return fx

# Interfaz


In [75]:
layout = [
            [sg.Text('Seleccione un archivo'),sg.Input(), sg.FileBrowse('Buscar')],
            [sg.Text('Seleccione un problema')],
            [sg.Radio('Mochila', "problema"), sg.Radio('Viajero', "problema"),sg.Radio('Minimos cuadrados', "problema", default=True)],
            [sg.Text('Seleccione el operador de selección'),sg.Text('Seleccione el operador de cruzamiento'),sg.Text('Seleccione el operador de mutacion')],
            [sg.InputCombo(('NAM', 'proporcional', 'torneo'),size=(25,1),default_value="proporcional"),sg.InputCombo(('cruzaNPuntos', 'cruzaUniforme', 'cruzaOrdenada', 'cruza_aritmetica'),size=(25,1),default_value="cruza_aritmetica"),sg.InputCombo(('swap', 'swap_por_gen', 'sim', 'sim_por_gen', 'uniform_mutation', 'boundary_mutation', 'mutacion_bin'),size=(25,1),default_value="uniform_mutation")],
            [sg.Text('Escriba la cantidad de generaciones'),sg.InputText(key='generaciones',default_text = 500)],
            [sg.Text('Escriba la probabilidad de cruzamiento'),sg.InputText(key='cruzamiento',default_text = 0.9)],
            [sg.Text('Escriba la probabilidad de mutación'),sg.InputText(key='mutation',default_text = 0.05),],
            [sg.Text('Escriba el peso de la mochila'), sg.InputText(key='peso',default_text = 0)],
            [],
            [sg.Text('Dimension (problema de minimos cuadrados)'),sg.InputText(key='dimension',default_text = 30)],
            [],
            [sg.Text('Escriba el tamaño de la población'),sg.InputText(key='tam',default_text = 50)],
            [],
            [sg.Text('Seleccione la distancia para el operador NAM'),sg.InputCombo(('distanciaEuclidiana', 'distanciaHamming'),size=(25,1),default_value="distanciaEuclidiana")],
            [],
            [sg.Text('Escriba el numero de padres para el operador de seleccion'),sg.InputText(key='padres', default_text = 2)],
            [],
            [sg.Text('Escriba el tamaño del torneo'),sg.InputText(key='torneo', default_text = 6)],
            [],
            [sg.Text('Escriba el numero de puntos para el operador de cruza'),sg.InputText(key='npuntos', default_text = 2)],
            [],
            [sg.Text('Escriba el numero de hijos para el operador de cruza'),sg.InputText(key='nhijos', default_text = 2)],
            [],
            [sg.Text('Escriba rango inferior de la funcion'),sg.InputText(key='rangoInf', default_text = -10)],
            [],
            [sg.Text('Escriba rango superior de la funcion'),sg.InputText(key='rangoSup', default_text = 10)],
            [],
            [sg.Radio('Maximizar', "maximizar"), sg.Radio('Minimizar', "maximizar", default=True)],
            [sg.Radio('Generacional', "generacional"), sg.Radio('Estacionario', "generacional", default=True)],
            [sg.Text('Seleccione operador de reemplazo'),sg.InputCombo(('reemplazo_rand', 'reemplazo_CD', 'TRS'),size=(25,1),default_value="reemplazo_CD")],
            [sg.Text('Seleccione la función a evaluar'),sg.InputCombo(('Alpine1', 'Dixon', 'Quintic','Schwefel','Streched','SumSquares'),size=(25,1),default_value="Alpine1"),],
            [sg.OK(), sg.Cancel()]
        ]



window = sg.Window('Get filename example', layout)
event, values = window.read()
window.close()

In [76]:
values



{0: None,
 'Buscar': None,
 1: None,
 2: None,
 3: None,
 4: None,
 5: None,
 6: None,
 'generaciones': None,
 'cruzamiento': None,
 'mutation': None,
 'peso': None,
 'dimension': None,
 'tam': None,
 7: None,
 'padres': None,
 'torneo': None,
 'npuntos': None,
 'nhijos': None,
 'rangoInf': None,
 'rangoSup': None,
 8: None,
 9: None,
 10: None,
 11: None,
 12: None,
 13: None}

# Generacional


In [77]:
def generacional(poblacion, evaluaciones, operador_seleccion, operador_cruza, operador_mutacion, probabilidad_mutacion, distancia, numero_padres, tam_torneo, puntos, nhijos, rangoInf, rangoSup, prob_cruza, generaciones,mochila,pesos = 0,pesoM=0,maximizar=True):
    
    nueva_poblacion = []
    
    while len(nueva_poblacion) < len(poblacion):
            #print(operador_seleccion,poblacion, evaluaciones, str(distancia), numero_padres, tam_torneo )
            #(poblacion, evaluaciones, funcionDistancia = 0, sujetos = 0, numero_padres = 0, tam_torneo = 0)
            
            # ------------------------------------
            #        Operador de selección
            # ------------------------------------
            padres = eval(operador_seleccion + '(poblacion, evaluaciones, distancia, numero_padres, tam_torneo, maximizar)')
            #(padres, puntos = 0, num_hijos=0)
            #print(padres)
            
            # ------------------------------------
            #       Operador de cruzamiento
            # ------------------------------------
            cruza_prob = random.uniform(0,1)
            
            if cruza_prob <= prob_cruza:
                hijo_verificado = 0
                while hijo_verificado < nhijos:
                    hijo_verificado = 0
                    hijos = eval(operador_cruza + '(padres, puntos, nhijos)')
                    if mochila == True:
                        for hijo in hijos:
                            pesoT = PesoTotal(hijo,pesos,pesoM)
                            if pesoT == True:
                                hijo_verificado = hijo_verificado + 1
                    else:
                        hijo_verificado = nhijos

            # Operador de Mutación    
                for hijo in hijos:
                    if rango[1] == 0 or rango[0] == 0:
                        if mochila == True:
                            pesoT = False
                            while pesoT == False:
                                hijo_mutado = eval(operador_mutacion+ '(hijo,probabilidad_mutacion)')
                                pesoT = PesoTotal(hijo_mutado,pesos,pesoM)
                            nueva_poblacion.append(hijo_mutado)
                        else:
                            nueva_poblacion.append(eval(operador_mutacion+ '(hijo,probabilidad_mutacion)'))
                    else:
                        nueva_poblacion.append(eval(operador_mutacion + '(hijo,rango,probabilidad_mutacion)'))
    return nueva_poblacion

# Estacionario

In [78]:
def estacionario(poblacion, evaluaciones, operador_seleccion, operador_cruza, operador_mutacion, probabilidad_mutacion, distancia, numero_padres, tam_torneo, puntos, nhijos, rangoInf, rangoSup, prob_cruza, generaciones,mochila,pesos = 0,pesoM=0,maximizar=True):
    
    nuevos_hijos = []
    bandera = 0
    while bandera == 0:
        # ------------------------------------
        #        Operador de selección
        # ------------------------------------
        padres = eval(operador_seleccion + '(poblacion, evaluaciones, distancia, numero_padres, tam_torneo, maximizar)')
        #(padres, puntos = 0, num_hijos=0)
        #print(padres)

        # ------------------------------------
        #       Operador de cruzamiento
        # ------------------------------------
        cruza_prob = random.uniform(0,1)

        if cruza_prob <= prob_cruza:
            hijo_verificado = 0
            while hijo_verificado < nhijos:
                hijo_verificado = 0
                hijos = eval(operador_cruza + '(padres, puntos, nhijos)')
                if mochila == True:
                    for hijo in hijos:
                        pesoT = PesoTotal(hijo,pesos,pesoM)
                        if pesoT == True:
                            hijo_verificado = hijo_verificado + 1
                else:
                    hijo_verificado = nhijos

        # Operador de Mutación    
            for hijo in hijos:
                if rango[1] == 0 or rango[0] == 0:
                    if mochila == True:
                        pesoT = False
                        while pesoT == False:
                            hijo_mutado = eval(operador_mutacion+ '(hijo,probabilidad_mutacion)')
                            pesoT = PesoTotal(hijo_mutado,pesos,pesoM)
                        nuevos_hijos.append(hijo_mutado)
                    else:
                        nuevos_hijos.append(eval(operador_mutacion+ '(hijo,probabilidad_mutacion)'))
                else:
                    nuevos_hijos.append(eval(operador_mutacion + '(hijo,rango,probabilidad_mutacion)'))
            bandera = 1

    return nuevos_hijos, padres

In [79]:
def val_unicos(lista):
    unicos = []

    for i in lista:
        if i not in unicos:
            unicos.append(i)

    return unicos

#### Agregar que solo evalue cierto numero de evaluaciones para competencia

In [80]:
if values[1] == True: #mochila
    
    file = open(values[0], 'r')
    archivo = pandas.read_csv(file, sep=',')
    pesos = archivo.P.tolist()
    valores = archivo.V.tolist()
    filas = len(archivo)
    
    poblacion = poblacionMochila(int(values['tam']),filas,pesos,int(values['peso']))
    
    rango = [int(values['rangoInf']),int(values['rangoSup'])]
    print(poblacion)
    
    generacion = 0
    while generacion < int(values['generaciones']):
        if values[10] == True:
            evaluaciones = evaluacionMochila(poblacion, valores)
            # mejor individuo

            nueva_poblacion = generacional(poblacion, evaluaciones, (values[4]), (values[5]), (values[6]), float(values['mutation']), (values[7]), int(values['padres']), int(values['torneo']), int(values['npuntos']), int(values['nhijos']), int(values['rangoInf']), int(values['rangoSup']),float(values['cruzamiento']),int(values['generaciones']),values[1],pesos,int(values['peso']),values[8])
            nueva_poblacion = np.array(nueva_poblacion)
            # buscar y guardar mejor individuo de poblacion actual y en que generacion salio
            # determinar si al final devolvemos el mejor de todas las generaciones o el mejor de la ultima generación
            poblacion = copy.copy(nueva_poblacion)
            generacion = generacion + 1
            print(generacion)
        else:
            evaluaciones = evaluacionMochila(poblacion, valores)
            # Regresa hijos
            nuevos_hijos, padres = estacionario(poblacion, evaluaciones, (values[4]), (values[5]), (values[6]), float(values['mutation']), (values[7]), int(values['padres']), int(values['torneo']), int(values['npuntos']), int(values['nhijos']), int(values['rangoInf']), int(values['rangoSup']),float(values['cruzamiento']),int(values['generaciones']),values[1],pesos,int(values['peso']),values[8])
            # ------------------------------------
            #       Operador de reemplazo
            # ------------------------------------
            tamanio_t = int(values['torneo'])
            print('nuevos padres',padres)
            print('nuevos hijos',nuevos_hijos)
            poblacion = eval(values[12]+ '(poblacion,nuevos_hijos,valores,padres,ind_mochila,values[7],tamanio_t)')
            generacion = generacion + 1
            print('generacion',generacion)

    print(poblacion)
    print(evaluaciones) 
   

elif values[2] == True: #viajero
    
    file = open(values[0], 'r')
    archivo = pandas.read_csv(file, sep=',')
    ciudades1 = archivo.C1.tolist()
    ciudades2 = archivo.C2.tolist()
    distancia = archivo.D.tolist()
    ciudades_unicas = val_unicos(ciudades1)

    filas = len(ciudades_unicas)
    columnas = len(ciudades_unicas)
    
    matriz_ady = 500*np.ones((filas, columnas))

    for i in range(len(ciudades1)):
        matriz_ady[ciudades1[i] - 1, ciudades2[i] - 1] = distancia[i]
    
    poblacion = poblacionViajero(int(values['tam']),ciudades_unicas)
    rango = [int(values['rangoInf']),int(values['rangoSup'])]
    

    generacion = 0
    while generacion < int(values['generaciones']):
        if values[10] == True:
            print(generacion)
            evaluaciones = evaluacionViajero (poblacion, matriz_ady)
            nueva_poblacion = generacional(poblacion, evaluaciones, (values[4]), (values[5]), (values[6]), float(values['mutation']), (values[7]), int(values['padres']), int(values['torneo']), int(values['npuntos']), int(values['nhijos']), int(values['rangoInf']), int(values['rangoSup']),float(values['cruzamiento']),int(values['generaciones']),values[1],pesos,int(values['peso']),values[8])
            nueva_poblacion = np.array(nueva_poblacion)
            poblacion = copy.copy(nueva_poblacion)
            generacion = generacion + 1
        else:
            evaluaciones = evaluacionViajero(poblacion, matriz_ady)
            # Regresa hijos
            nuevos_hijos, padres = estacionario(poblacion, evaluaciones, (values[4]), (values[5]), (values[6]), float(values['mutation']), (values[7]), int(values['padres']), int(values['torneo']), int(values['npuntos']), int(values['nhijos']), int(values['rangoInf']), int(values['rangoSup']),float(values['cruzamiento']),int(values['generaciones']),values[1],pesos,int(values['peso']),values[8])
            # ------------------------------------
            #       Operador de reemplazo
            # ------------------------------------
            tamanio_t = int(values['torneo'])
            print('nuevos padres',padres)
            print('nuevos hijos',nuevos_hijos)
            poblacion = eval(values[12]+ '(poblacion,nuevos_hijos,matriz_ady,padres,ind_viajero,values[7],tamanio_t)')
            generacion = generacion + 1
            print('generacion',generacion)


    print(poblacion)
    print(evaluaciones) 
    
    
elif values[3] == True: #minimos
    
    tiempos = []
    evaluaciones_t = []
    
    start_time = time.time()
    for i in range(20):
        
        print(i)
        start_time = time.time()
        rango = [int(values['rangoInf']),int(values['rangoSup'])]
        poblacion = poblacionMinimos(int(values['tam']),int(values['dimension']),rango)
        #print(poblacion)
        evaluaciones = []
        pesos = 0
        mejor = 10000000000000000
        mejor_generacion = 0
        mejor_respuesta = []
        imax = 0
        peor_respuesta = []
        peor_generacion = 0
        promedio = 0

        generacion = 0
        while generacion < int(values['generaciones']) and imax < int(values['generaciones']):
            if values[10] == True:
                for i in range(len(poblacion)-1):
                    ciudadano = poblacion[i]
                    evaluacion = eval(values[13]+'(ciudadano)')
                    # Mejor global
                    if evaluacion < mejor:
                        mejor_respuesta = ciudadano
                        mejor = evaluacion
                        mejor_generacion = generacion
                    evaluaciones.append(evaluacion)
                nueva_poblacion = generacional(poblacion, evaluaciones, (values[4]), (values[5]), (values[6]), float(values['mutation']), (values[7]), int(values['padres']), int(values['torneo']), int(values['npuntos']), int(values['nhijos']), int(values['rangoInf']), int(values['rangoSup']),float(values['cruzamiento']),int(values['generaciones']),values[1],pesos,int(values['peso']),values[8])
                nueva_poblacion = np.array(nueva_poblacion)
                poblacion = copy.copy(nueva_poblacion)

                imax = imax + (len(poblacion)-1)
                generacion = generacion + 1
            else:
                for i in range(len(poblacion)-1):
                    ciudadano = poblacion[i]
                    evaluacion = eval(values[13]+'(ciudadano)')
                    # Mejor global
                    if evaluacion < mejor:
                        mejor_respuesta = ciudadano
                        mejor = evaluacion
                        mejor_generacion = generacion
                    evaluaciones.append(evaluacion)
                # Regresa hijos
                nuevos_hijos, padres = estacionario(poblacion, evaluaciones, (values[4]), (values[5]), (values[6]), float(values['mutation']), (values[7]), int(values['padres']), int(values['torneo']), int(values['npuntos']), int(values['nhijos']), int(values['rangoInf']), int(values['rangoSup']),float(values['cruzamiento']),int(values['generaciones']),values[1],pesos,int(values['peso']),values[8])
                # ------------------------------------
                #       Operador de reemplazo
                # ------------------------------------
                tamanio_t = int(values['torneo'])
                poblacion = eval(values[12]+ '(poblacion,nuevos_hijos,evaluaciones,padres,ind_minimos,values[7],tamanio_t)')
                imax = generacion
                generacion = generacion + 1

        tiempo = time.time() - start_time
        tiempos.append(tiempo)
        evaluaciones_t.append(evaluaciones)
        
        #print(poblacion)
        #print('Mejor resultado',mejor_respuesta)
        #print('Mejor evaluacion',mejor)
        #print('Generacion donde esta: ', mejor_generacion)
        #print('Imax', imax)
        #ultimas = []
        #for i in range(len(poblacion)-1):
        #            ciudadano = poblacion[i]
        #            evaluacion = eval(values[13]+'(ciudadano)')
        #            ultimas.append(evaluacion)
        #print('Ultimas evaluaciones', ultimas)
    evaluaciones_t2 = []
    for lista1 in evaluaciones_t:
        for numero in lista1:
            evaluaciones_t2.append(numero)
    evaluaciones_t2 = sorted(evaluaciones_t2)
    tiempos = sorted(tiempos)
    mediana = np.round(len(evaluaciones_t2)/2)
    mediana = mediana.astype(int)
    
    #----------
    mediana_tiempos = np.round(len(tiempos)/2).astype(int)
    
    
    print('-----------Resultados------------')
    print('Mejor  Peor  Promedio  Mediana  Desviacion')
    print(np.min(evaluaciones_t2),',',np.max(evaluaciones_t2),',',np.mean(evaluaciones_t2),',',evaluaciones_t2[mediana],',',np.std(evaluaciones_t2))
    print('-----------Tiempos---------------')
    print('Mejor       Peor      Promedio        Mediana      Desviacion')
    print(np.min(tiempos),',',np.max(tiempos),',',np.mean(tiempos),',',tiempos[mediana_tiempos],',',np.std(tiempos))
